# Autoencodeurs RGB
Le principe ici est d'utiliser le même type d'autoencodeur que pour une image en noir et blanc mais en l'utilisant 3 fois séparément sur chacune des couches RGB d'une image en couleur

## Installaton et Importations

In [ ]:
! pip install tensorflow

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

## Importation et prétraitement des données 

In [ ]:
import glob
from PIL import Image
filelist = glob.glob('/Clean_apple/*.jpg') #Liste des images contenues dans le dossier "Clean_apple"

apples_R = []
apples_G = []
apples_B = []

for i in range(472):
  img = Image.open(filelist[i])
  rgb = img.split()
  apples_R.append(np.asarray(rgb[0].resize((256,256))))
  apples_G.append(np.asarray(rgb[1].resize((256,256))))
  apples_B.append(np.asarray(rgb[2].resize((256,256))))

apples_R = np.asarray(apples_R)
apples_G = np.asarray(apples_G)
apples_B = np.asarray(apples_B)

## Création des jeux de données d'entraînements et de tests

Rouge

In [ ]:
x_train_R, x_test_R = apples_R[:400],apples_R[400:]

x_train_R = x_train_R.astype('float32') / 255.
x_test_R = x_test_R.astype('float32') / 255.

print (x_train_R.shape)
print (x_test_R.shape)

Vert

In [ ]:
x_train_G, x_test_G = apples_G[:400],apples_G[400:]

x_train_G = x_train_G.astype('float32') / 255.
x_test_G = x_test_G.astype('float32') / 255.

print (x_train_G.shape)
print (x_test_G.shape)

Bleu

In [ ]:
x_train_B, x_test_B = apples_B[:400],apples_B[400:]

x_train_B = x_train_B.astype('float32') / 255.
x_test_B = x_test_B.astype('float32') / 255.

print (x_train_B.shape)
print (x_test_B.shape)

## Création des modèles
Remarque : On a besoin que d'un seul modèle de départ qui est dupliqué et entrainé séparément pour chaque couleur

In [ ]:
latent_dim = 2048

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(65536, activation='sigmoid'),
      layers.Reshape((256, 256))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


Les 3 Autoencodeurs :

In [ ]:
autoencoder_R = Autoencoder(latent_dim) 
autoencoder_G = Autoencoder(latent_dim) 
autoencoder_B = Autoencoder(latent_dim) 

## Définition des checkpoints

In [ ]:
# R
checkpoint_dir_R = '/Autoencoder/R' 
checkpoint_prefix_R = os.path.join(checkpoint_dir_R, "ckpt")
checkpoint_R = tf.train.Checkpoint(autoencoder=autoencoder_R)

In [ ]:
# G
checkpoint_dir_G = '/Autoencoder/G' 
checkpoint_prefix_G = os.path.join(checkpoint_dir_G, "ckpt")
checkpoint_G = tf.train.Checkpoint(autoencoder=autoencoder_G)

In [ ]:
# B
checkpoint_dir_B = '/Autoencoder/B' 
checkpoint_prefix_B = os.path.join(checkpoint_dir_B, "ckpt")
checkpoint_B = tf.train.Checkpoint(autoencoder=autoencoder_B)

## Entrainements 

In [ ]:
autoencoder_R.compile(optimizer='adam', loss=losses.MeanSquaredError())
autoencoder_G.compile(optimizer='adam', loss=losses.MeanSquaredError())
autoencoder_B.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
# R
autoencoder_R.fit(x_train_R, x_train_R,
                epochs=200,
                shuffle=True,
                validation_data=(x_test_R, x_test_R))
checkpoint_R.save(file_prefix = checkpoint_prefix_R)

Epoch 1/200
13/13 [==============================] - 4s 81ms/step - loss: 0.0923 - val_loss: 0.0875
Epoch 2/200
13/13 [==============================] - 1s 70ms/step - loss: 0.0704 - val_loss: 0.0770
Epoch 3/200
13/13 [==============================] - 1s 71ms/step - loss: 0.0621 - val_loss: 0.0656
Epoch 4/200
13/13 [==============================] - 1s 68ms/step - loss: 0.0574 - val_loss: 0.0613
Epoch 5/200
13/13 [==============================] - 1s 70ms/step - loss: 0.0551 - val_loss: 0.0582
Epoch 6/200
13/13 [==============================] - 1s 70ms/step - loss: 0.0522 - val_loss: 0.0581
Epoch 7/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0527 - val_loss: 0.0607
Epoch 8/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0531 - val_loss: 0.0551
Epoch 9/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0487 - val_loss: 0.0545
Epoch 10/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0485 - val_loss: 0.0518

In [ ]:
# G
autoencoder_G.fit(x_train_G, x_train_G,
                epochs=200,
                shuffle=True,
                validation_data=(x_test_G, x_test_G))
checkpoint_G.save(file_prefix = checkpoint_prefix_G)

Epoch 1/200
13/13 [==============================] - 1s 80ms/step - loss: 0.1100 - val_loss: 0.1037
Epoch 2/200
13/13 [==============================] - 1s 70ms/step - loss: 0.0953 - val_loss: 0.0920
Epoch 3/200
13/13 [==============================] - 1s 70ms/step - loss: 0.0829 - val_loss: 0.0798
Epoch 4/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0752 - val_loss: 0.0735
Epoch 5/200
13/13 [==============================] - 1s 68ms/step - loss: 0.0687 - val_loss: 0.0680
Epoch 6/200
13/13 [==============================] - 1s 70ms/step - loss: 0.0631 - val_loss: 0.0638
Epoch 7/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0597 - val_loss: 0.0600
Epoch 8/200
13/13 [==============================] - 1s 70ms/step - loss: 0.0551 - val_loss: 0.0599
Epoch 9/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0514 - val_loss: 0.0620
Epoch 10/200
13/13 [==============================] - 1s 70ms/step - loss: 0.0507 - val_loss: 0.0550

In [ ]:
# B
autoencoder_B.fit(x_train_B, x_train_B,
                epochs=200,
                shuffle=True,
                validation_data=(x_test_B, x_test_B))
checkpoint_B.save(file_prefix = checkpoint_prefix_B)

Epoch 1/200
13/13 [==============================] - 1s 82ms/step - loss: 0.1140 - val_loss: 0.1143
Epoch 2/200
13/13 [==============================] - 1s 68ms/step - loss: 0.1010 - val_loss: 0.0951
Epoch 3/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0836 - val_loss: 0.0778
Epoch 4/200
13/13 [==============================] - 1s 68ms/step - loss: 0.0712 - val_loss: 0.0738
Epoch 5/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0632 - val_loss: 0.0729
Epoch 6/200
13/13 [==============================] - 1s 68ms/step - loss: 0.0588 - val_loss: 0.0620
Epoch 7/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0537 - val_loss: 0.0596
Epoch 8/200
13/13 [==============================] - 1s 68ms/step - loss: 0.0485 - val_loss: 0.0552
Epoch 9/200
13/13 [==============================] - 1s 70ms/step - loss: 0.0445 - val_loss: 0.0572
Epoch 10/200
13/13 [==============================] - 1s 69ms/step - loss: 0.0413 - val_loss: 0.0523

## Résultats de l'entraînement

In [ ]:
encoded_imgs_R = autoencoder_R.encoder(x_test_R).numpy()
decoded_imgs_R = autoencoder_R.decoder(encoded_imgs_R).numpy()

encoded_imgs_G = autoencoder_G.encoder(x_test_G).numpy()
decoded_imgs_G = autoencoder_G.decoder(encoded_imgs_G).numpy()

encoded_imgs_B = autoencoder_B.encoder(x_test_B).numpy()
decoded_imgs_B = autoencoder_B.decoder(encoded_imgs_B).numpy()

Résultats pour l'autoencodeur rouge :

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test_R[i], "Reds_r")
  plt.title("original_R")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs_R[i], "Reds_r")
  plt.title("reconstructed_R")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()


Résultats pour l'autoencodeur vert :

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test_G[i], "Greens")
  plt.title("original_G")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs_G[i], "Greens")
  plt.title("reconstructed_G")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

Résultats pour l'autoencodeur bleu :

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test_B[i], "Blues")
  plt.title("original_B")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs_B[i], "Blues")
  plt.title("reconstructed_B")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

Fusion des résultats :

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)  
  plt.imshow(np.dstack([x_test_R[i], x_test_G[i],x_test_B[i]]))
  plt.title("original")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(np.dstack([decoded_imgs_R[i], decoded_imgs_G[i],decoded_imgs_B[i]]))
  plt.title("reconstructed")
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()
